In [ ]:
import matplotlib.pyplot as plt

def simulate_scheduling(num_pods, cpu_request, cpu_limit, nodes):
    """
    Simulate scheduling pods for a single namespace.
    
    Parameters:
      - num_pods: Total number of pods to schedule.
      - cpu_request: CPU request for each pod.
      - cpu_limit: CPU limit for each pod.
      - nodes: A list of tuples (node_name, total_cpu_capacity)
      
    Returns:
      - A dictionary mapping node names to the number of pods scheduled.
      - A dictionary mapping node names to the total CPU limits allocated.
      - A dictionary mapping node names to the total CPU requests allocated.
    """
    # Initialize node state for each node.
    state = {name: {"pods": 0, "allocated_limits": 0, "allocated_requests": 0} for name, cap in nodes}
    
    for _ in range(num_pods):
        best_node = None
        best_ratio = float('inf')
        
        # Evaluate each node for scheduling the new pod.
        for name, capacity in nodes:
            # Check available capacity based on CPU requests.
            allocated_req = state[name]["allocated_requests"]
            if allocated_req + cpu_request > capacity:
                continue  # Not enough free capacity.
            
            # Calculate ratio: (current allocated CPU limits in this namespace + new pod's limit) / total capacity.
            allocated_limits = state[name]["allocated_limits"]
            ratio = (allocated_limits + cpu_limit) / capacity
            
            # Choose node with lowest ratio.
            if ratio < best_ratio:
                best_ratio = ratio
                best_node = name
                
        # If no node could accommodate the pod, print a warning and break.
        if best_node is None:
            print("Warning: No node has sufficient capacity for the new pod.")
            break
        
        # Schedule the pod on the chosen node.
        state[best_node]["pods"] += 1
        state[best_node]["allocated_limits"] += cpu_limit
        state[best_node]["allocated_requests"] += cpu_request
    
    # Prepare results dictionaries.
    pods_per_node = {name: info["pods"] for name, info in state.items()}
    limits_per_node = {name: info["allocated_limits"] for name, info in state.items()}
    requests_per_node = {name: info["allocated_requests"] for name, info in state.items()}
    
    return pods_per_node, limits_per_node, requests_per_node

def plot_distribution(pods_per_node):
    nodes = list(pods_per_node.keys())
    pods = [pods_per_node[node] for node in nodes]
    
    plt.figure(figsize=(16, 5))
    bars = plt.bar(nodes, pods, color="skyblue")
    plt.xlabel("Node")
    plt.ylabel("Number of Pods Scheduled")
    plt.title("Pod Distribution per Node")
    
    # Annotate the bars with the pod count.
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# Define simulation parameters directly
num_pods = 120
cpu_request = 1  # Default CPU request per pod
cpu_limit = 8    # Default CPU limit per pod

# Define nodes as a list of tuples: (node_name, total_cpu_capacity)
# In this simulation, we have 4 nodes: three with 64 CPUs and one with 128 CPUs.
nodes = [("node1", 128), ("node2",128), ("node3", 128), ("node4", 128), ("node5", 192), ("node6", 192)]

# Run the simulation.
pods_per_node, limits_per_node, requests_per_node = simulate_scheduling(num_pods, cpu_request, cpu_limit, nodes)

print("Pods scheduled per node:", pods_per_node)
print("Total CPU limits allocated per node:", limits_per_node)
print("Total CPU requests allocated per node:", requests_per_node)

# Plot the pod distribution
plot_distribution(pods_per_node)
